In [1]:
import json
import jsonlines
import pyspark.pandas
from pyspark import SparkContext
import pyspark

/home/sebastien/miniconda3/envs/2023/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
sc = SparkContext()

23/11/15 18:54:24 WARN Utils: Your hostname, sebastien-Inspiron-7391-2n1 resolves to a loopback address: 127.0.1.1; using 192.168.1.73 instead (on interface wlp0s20f3)
23/11/15 18:54:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/15 18:54:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
path_to_disk = '/Volumes/Maxtor'
path_to_disk_ubuntu = '/media/sebastien/Maxtor'
file_path = '/yt_metadata_en.jsonl'

# pyspark_df = pyspark.pandas.read_json(path_to_disk_ubuntu + file_path)
raw_data = sc.textFile(path_to_disk_ubuntu + file_path)
video_dataset = raw_data.map(lambda x: json.loads(x))
# video_dataset = sc.parallelize(raw_data.take(1000)).map(lambda x: json.loads(x))
video_dataset = video_dataset.filter(lambda x: x['view_count'] is not None and x['upload_date'] is not None)

In [4]:
views_per_date = video_dataset.map(lambda x: (x['upload_date'], x['view_count']))
views_per_date = views_per_date.groupBy(lambda x: x[0]).map(lambda x: (x[0], len(x[1]), sum(map(lambda y: y[1], x[1])))).collect()

In [5]:
videos_by_date_path = "data/nb_videos_and_views_by_date.jsonl"
with jsonlines.open(videos_by_date_path, "w") as jsonl_file:
    jsonl_file.write_all(views_per_date)
sc.stop()